In [1]:
%matplotlib inline

import numpy as np
import pandas as pd
import os
import dicom
import scipy.ndimage
import matplotlib.pyplot as plt
import time
import traceback
import tensorflow as tf
import numpy as np
import scipy.ndimage

from multiprocessing.dummy import Pool as ThreadPool 
from tqdm import tqdm
from skimage import measure, morphology
from skimage.draw import polygon
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
from tempfile import TemporaryFile

# Get a list of patient folders in the data folder.
INPUT_FOLDER = 'example_dcm/'
patients = os.walk('./example_dcm').next()[1]
patients.sort()

test_patients = os.walk('./provisional_dcm_no_gt').next()[1]
test_patients.sort()

In [2]:
IMG_SIZE_PX = 50
SLICE_COUNT = 20

n_classes = 6

x = tf.placeholder('float')
y = tf.placeholder('float')

keep_rate = 0.8

In [3]:
def get_bounding_cube(cancer, resize_factor):
    a = np.where(cancer != 0)
    coords = np.min(a[0]), np.max(a[0]), np.min(a[1]), np.max(a[1]), np.min(a[2]), np.max(a[2])
    box = [coords[0] * resize_factor[0], coords[2] * resize_factor[1], coords[4] * resize_factor[2], (coords[1] - coords[0]) * resize_factor[0], (coords[3] - coords[2]) * resize_factor[1], (coords[5] - coords[4]) * resize_factor[2]]
    return box

In [4]:
# Load the CT and RT documents for a patient.
def load_scan(path):
    def listdir_nohidden(path):
        for f in os.listdir(path):
            if not f.startswith('.'):
                yield f
    slice_rt = [dicom.read_file(path + "/RTst/" + s) for s in listdir_nohidden(path + "/RTst")]
    slice_ct = [dicom.read_file(path + "/CT/" + s) for s in listdir_nohidden(path + "/CT")]
    slice_ct.sort(key = lambda x: int(x.ImagePositionPatient[2]))    
    return slice_ct, slice_rt

In [5]:
def get_image_patient(patient):
    if os.path.exists(INPUT_FOLDER + patient + "/50x50x20.npy"):
        if os.path.exists(INPUT_FOLDER + patient + "/50x50x20_cancer.npy"):
            resized_image = np.load(INPUT_FOLDER + patient + "/50x50x20.npy");
            cancer_cube = np.load(INPUT_FOLDER + patient + "/50x50x20_cancer.npy");
            return resized_image, cancer_cube / 50;
        
    image = np.load(INPUT_FOLDER + patient + "/image.npy");
    cancer = np.load(INPUT_FOLDER + patient + "/truthV1.npy");
    resize_factor = [float(IMG_SIZE_PX) / image.shape[0], float(IMG_SIZE_PX) / image.shape[1], float(SLICE_COUNT) / image.shape[2]]
    resized_image = scipy.ndimage.interpolation.zoom(image, resize_factor, mode='nearest');
    cancer_cube = get_bounding_cube(cancer, 1 / image.shape);
    np.save(INPUT_FOLDER + patient + "/50x50x20.npy", resized_image);
    np.save(INPUT_FOLDER + patient + "/50x50x20_cancer.npy", cancer_cube);
    
    # Uncomment to show image of patient CT somewhere in the middle
    # plt.imshow(resized_image.transpose()[30])
    # plt.show()
            
    return resized_image, cancer_cube 

In [6]:
def conv3d(x, W):
    return tf.nn.conv3d(x, W, strides=[1,1,1,1,1], padding='SAME')

def maxpool3d(x):
    #                        size of window         movement of window as you slide about
    return tf.nn.max_pool3d(x, ksize=[1,2,2,2,1], strides=[1,2,2,2,1], padding='SAME')
def fc(x, weight, bias):
    shape = x.get_shape().as_list()
    dim = 1
    for d in shape[1:]:
        if d is not None:
            dim *= d
    x = tf.reshape(x, [-1, dim])
    return tf.matmul(x, weight) + bias

In [7]:
def conv_2(x):
               # 3 x 3 x 3 patches, 1 channel, 32 features to compute.
    weights = {'W_conv1':tf.Variable(tf.random_normal([3, 3, 3, 1, 32])),
               # 3 x 3 x 3 patches, 32 channels, 64 features to compute.
               'W_conv2':tf.Variable(tf.random_normal([3, 3, 3, 32, 64])),
               # 52 because 50 plus zero padding on each border
               'W_fc':tf.Variable(tf.random_normal([(IMG_SIZE_PX + 2) * (IMG_SIZE_PX + 2) * SLICE_COUNT, 1024])),
               'out':tf.Variable(tf.random_normal([1024, n_classes]))}

    biases = { 'b_conv1':tf.Variable(tf.random_normal([32])),
               'b_conv2':tf.Variable(tf.random_normal([64])),
               'b_fc':tf.Variable(tf.random_normal([1024])),
               'out':tf.Variable(tf.random_normal([n_classes], 0, 0.1))}

    #                            image X      image Y        image Z
    x = tf.reshape(x, shape=[-1, IMG_SIZE_PX, IMG_SIZE_PX, SLICE_COUNT, 1])

    conv1 = tf.nn.relu(conv3d(x, weights['W_conv1']) + biases['b_conv1'])
    conv1 = maxpool3d(conv1)


    conv2 = tf.nn.relu(conv3d(conv1, weights['W_conv2']) + biases['b_conv2'])
    conv2 = maxpool3d(conv2)

    fc = tf.reshape(conv2,[-1, (IMG_SIZE_PX + 2) * (IMG_SIZE_PX + 2) * SLICE_COUNT])
    fc = tf.nn.relu(tf.matmul(fc, weights['W_fc']) + biases['b_fc'])
    fc = tf.nn.dropout(fc, keep_rate)

    output = tf.matmul(fc, weights['out']) + biases['out']

    return output

In [8]:
def pre_process_all_images(patients):
    for i in patients:
        print "Resizing patient ", i
        resized_image, cube = get_image_patient(i)

In [9]:
def test_neural_network(x):
    prediction = conv_2(x)
    cost = tf.reduce_sum(tf.pow(prediction - y, 2))
    optimizer = tf.train.AdamOptimizer(learning_rate=1e-3).minimize(cost)
    
    tf.add_to_collection('optimizer', optimizer)
    tf.add_to_collection('cost', cost)
    
    saver = tf.train.Saver()
    
    sess = tf.Session();
    
    sess.run(tf.global_variables_initializer())

    if os.path.exists("model-19.meta"):
        saver = tf.train.import_meta_graph('./model-19.meta')
        saver.restore(sess, './model-19')
        optimizer = tf.get_collection('optimizer')[0]
        cost = tf.get_collection('cost')[0]
        for i in patients[:-50]:
            resized_image, cube = get_image_patient(i)
            _, c = sess.run([optimizer, cost], feed_dict={x: resized_image, y: cube})
        print("Model restored.")
        
        final_boxes = [];
        for i in patients[:-50]:
            a,b = get_image_patient(i)
            final_boxes.append(sess.run(prediction, feed_dict={x:a}));
            
        return final_boxes;

In [10]:
# pre_process_all_images(patients)
test_neural_network(x)

Model restored.


[array([[ 0.39529282,  0.44529739,  0.22401525,  0.09126683,  0.09195869,
          0.04262531]], dtype=float32),
 array([[ 0.39529282,  0.44529739,  0.22401525,  0.09126683,  0.09195869,
          0.04262531]], dtype=float32),
 array([[ 0.39529282,  0.44529739,  0.22401525,  0.09126683,  0.09195869,
          0.04262531]], dtype=float32),
 array([[ 0.39529282,  0.44529739,  0.22401525,  0.09126683,  0.09195869,
          0.04262531]], dtype=float32),
 array([[ 0.39529282,  0.44529739,  0.22401525,  0.09126683,  0.09195869,
          0.04262531]], dtype=float32),
 array([[ 0.39529282,  0.44529739,  0.22401525,  0.09126683,  0.09195869,
          0.04262531]], dtype=float32),
 array([[ 0.39529282,  0.44529739,  0.22401525,  0.09126683,  0.09195869,
          0.04262531]], dtype=float32),
 array([[ 0.39529282,  0.44529739,  0.22401525,  0.09126683,  0.09195869,
          0.04262531]], dtype=float32),
 array([[ 0.39529282,  0.44529739,  0.22401525,  0.09126683,  0.09195869,
          0.04